In [ ]:
import boto3
import csv
import os
import datetime
from collections import defaultdict

s3 = boto3.client("s3")


geo_dict = {
    4:"Andalucía", 5:"Aragón", 6:"Cantabria", 7:"Castilla la Mancha",
    8:"Castilla y León", 9:"Cataluña", 10:"País Vasco",
    11:"Principado de Asturias", 8744:"Comunidad de Ceuta",
    8745:"Comunidad de Melilla", 13:"Comunidad de Madrid",
    14:"Comunidad de Navarra", 15:"Comunidad Valenciana",
    16:"Extremadura", 17:"Galicia", 8743:"Islas Baleares",
    8742:"Islas Canarias", 20:"La Rioja", 21:"Región de Murcia"
}

BUCKET_NAME = "proyectohabree"
DATA_KEY = "demanda_evolucion.csv"  

def lambda_handler(event, context):
    hoy = datetime.date.today()
    prefix = f"demanda_diaria/{hoy}/"

    #Descargar CSV completo de S3
    local_csv = "/tmp/demanda_evolucion.csv"
    s3.download_file(BUCKET_NAME, DATA_KEY, local_csv)

    #Leer CSV y agrupar filas por comunidad
    grupos = defaultdict(list)
    with open(local_csv, newline='', encoding="utf-8") as f:
        reader = csv.DictReader(f)
        fieldnames = reader.fieldnames + ["comunidad"]  # añadir columna comunidad
        for row in reader:
            geo_id = int(row["geo_id"])
            comunidad = geo_dict.get(geo_id, "Desconocida")
            row["comunidad"] = comunidad
            grupos[comunidad].append(row)

    #Crear CSV por comunidad y subir a S3 
    for comunidad, filas in grupos.items():
        nombre = comunidad.replace(" ", "_") + ".csv"
        ruta_local = f"/tmp/{nombre}"
        ruta_s3 = prefix + nombre

        with open(ruta_local, "w", newline='', encoding="utf-8") as f_out:
            writer = csv.DictWriter(f_out, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(filas)

        s3.upload_file(ruta_local, BUCKET_NAME, ruta_s3)
        print(f"Subido: s3://{BUCKET_NAME}/{ruta_s3}")

    return {
        "statusCode": 200,
        "body": f"Archivos diarios guardados en s3://{BUCKET_NAME}/{prefix}"
    }
